## 📚 Assignment: Transfer Learning & The Power of Initialization
## Building Intuition for MAML

**Learning Objectives:**
- Understand why initialization matters for few-shot learning
- Experience the difference between various pre-training strategies
- Develop intuition for what MAML tries to optimize

**Advice on using LLM's**

---


Avoid it , but unfortunately we cannot stop you from using it , dont ask it everything more you think on your own the better , but whenever you take in a code from it , understand how that part fits in the current code , is there some optimization it did on its own, node it down or comment it in the code.

In [1]:
!pip install -q torch torchvision matplotlib numpy

#Understand what does each of this import do , see what all functions this hold
#whenever you want to implement something think which of this would you use and refer to its doc for the syntax

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, Subset
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from collections import Counter
import random

print("✅ Setup complete!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

✅ Setup complete!
PyTorch version: 2.9.0+cpu
CUDA available: False
Using device: cpu


## 📊 Part A: Dataset Preparation

We'll use **MNIST** for simplicity (or you can use Omniglot if you prefer).

**Your Task:**
- Split MNIST into 5 tasks (Tasks A-E), each with 2 digit classes
- For example: Task A = {0, 1}, Task B = {2, 3}, etc.

In [2]:


# Complete MNIST loading with standard transform and background note
transform = transforms.Compose([transforms.ToTensor(),  # Converts PIL image to tensor [0,1]
                                ])

train_dataset = torchvision.datasets.MNIST(
    root='./data', train=True, download=True, transform=transform
)
test_dataset = torchvision.datasets.MNIST(
    root='./data', train=False, download=True, transform=transform
)

print(f"✅ MNIST loaded: {len(train_dataset)} train, {len(test_dataset)} test images")

# Task definitions: 5 binary tasks with 2 digit classes each [web:3]
task_definitions = {
    'A': [0, 1],
    'B': [2, 3],
    'C': [4, 5],
    'D': [6, 7],
    'E': [8, 9]
}

def create_task_datasets(dataset, task_classes, n_train=15, n_support=5, n_query=10):
    """
    Creates train/support/query splits for a few-shot task from dataset.
    1) train: n_train total examples for task (background training)
    2) support: n_support per class (2*n_support total)
    3) query: n_query per class (2*n_query total)
    Assumes balanced classes; samples randomly without replacement.
    """
    # Get indices for each class in the dataset
    class_indices = [[] for _ in range(10)]
    for idx, label in enumerate(dataset.targets):
        class_indices[label].append(idx)

    cls1, cls2 = task_classes
    indices_cls1 = class_indices[cls1]
    indices_cls2 = class_indices[cls2]

    # Sample distinct indices for splits (shuffle for randomness)
    rng = np.random.default_rng()  # Or use random.seed()
    train_indices = (
        rng.choice(indices_cls1, min(n_train//2, len(indices_cls1)), replace=False).tolist() +
        rng.choice(indices_cls2, min(n_train//2, len(indices_cls2)), replace=False).tolist()
    )
    remain_cls1 = [i for i in indices_cls1 if i not in train_indices]
    remain_cls2 = [i for i in indices_cls2 if i not in train_indices]

    support_cls1 = rng.choice(remain_cls1, min(n_support, len(remain_cls1)), replace=False).tolist()
    remain_cls1 = [i for i in remain_cls1 if i not in support_cls1]
    support_cls2 = rng.choice(remain_cls2, min(n_support, len(remain_cls2)), replace=False).tolist()
    remain_cls2 = [i for i in remain_cls2 if i not in support_cls2]

    query_cls1 = rng.choice(remain_cls1, min(n_query, len(remain_cls1)), replace=False).tolist()
    query_cls2 = rng.choice(remain_cls2, min(n_query, len(remain_cls2)), replace=False).tolist()

    train_subset = Subset(dataset, train_indices)
    support_subset = Subset(dataset, support_cls1 + support_cls2)
    query_subset = Subset(dataset, query_cls1 + query_cls2)

    return train_subset, support_subset, query_subset

# Test the function
train_A, support_A, query_A = create_task_datasets(train_dataset, task_definitions['A'])
print(f"Task A - Train: {len(train_A)}, Support: {len(support_A)}, Query: {len(query_A)}")


100%|██████████| 9.91M/9.91M [00:01<00:00, 5.29MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 157kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.48MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.46MB/s]


✅ MNIST loaded: 60000 train, 10000 test images
Task A - Train: 14, Support: 10, Query: 20


Part A (continued): **Build Your Model**

**TODO:** Design a simple CNN for digit classification

Now since the model is ready we decide how do we want to train it :

First Do normal classification on large dataset of Task A - 0 & 1.

The we will do fine tuning

1.   Random Initialisation and then fine tune using support dataset, say we do this for task A which were 0 & 1 digits (save this)
2.   Take the above model weights and fine tune it on the support dataset for some other task , say B(2's & 3's)
3.   First train the model on all combined train dataset for all 10 digits(from all tasks A,B,C,D,E), then save it and then fine tune it on support dataset on to make a binary classifier , any 1 task say A here now digits will be classified. 0 class->0 digit , 1->1.

While moving from one model to other , think what layers do i need to keep and what do i need to remove.



In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)  # In:1x28x28 -> 32x28x28
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # -> 64x28x28
        self.pool = nn.MaxPool2d(2, 2)                           # Halves spatial dims
        self.fc1 = nn.Linear(64 * 7 * 7, 128)                    # After two pools: 64x7x7
        self.fc2 = nn.Linear(128, 10)                            # 10 digits
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))     # 32x14x14
        x = self.pool(F.relu(self.conv2(x)))     # 64x7x7
        x = x.view(-1, 64 * 7 * 7)              # Flatten
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)           # For NLLLoss


In [4]:
#Method 1 - Fully Trained Binary Classifier on Task A
# Binary model for Task A
class BinaryCNN(SimpleCNN):
    def __init__(self):
        super().__init__()
        self.fc2 = nn.Linear(128, 2)  # Change to 2 outputs

# Remap labels for binary: original label % 2 or map {0:0, 1:1}
def binary_collate(batch):
    images, labels = zip(*batch)
    images = torch.stack(images)
    labels = torch.tensor([1 if l == task_definitions['A'][1] else 0 for l in labels])
    return images, labels

train_loader_A = DataLoader(train_A, batch_size=64, shuffle=True, collate_fn=binary_collate)

model_A = BinaryCNN().to(device)
optimizer = optim.Adam(model_A.parameters(), lr=0.001)
criterion = nn.NLLLoss()  # Or CrossEntropyLoss
num_epochs=10
# Train loop (5-10 epochs)
for epoch in range(num_epochs):
    model_A.train()
    for imgs, labels in train_loader_A:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        out = model_A(imgs)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
torch.save(model_A.state_dict(), 'model_A.pth')  # Save full model
model_rand = BinaryCNN().to(device)  # Random init
optimizer = optim.Adam(model_rand.parameters(), lr=0.0001)  # Lower LR

support_loader_A = DataLoader(support_A, batch_size=32, shuffle=True, collate_fn=binary_collate)
# Train on support_A (small, few-shot) for 20-50 epochs similarly
torch.save(model_rand.state_dict(), 'model_rand_A.pth')


In [8]:
#Method 2 - Fine Tuning all 3 methods
# Load A weights into new BinaryCNN for B
model_B = BinaryCNN().to(device)
model_B.load_state_dict(torch.load('model_A.pth'), strict=False)  # Ignores fc2 mismatch

# Freeze early layers (optional, reduces catastrophic forgetting)
for param in model_B.conv1.parameters(): param.requires_grad = False
for param in model_B.conv2.parameters(): param.requires_grad = False
# Unfreeze fc1, fc2

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model_B.parameters()), lr=0.0001)

def binary_collate_B(batch):  # For Task B classes
    images, labels = zip(*batch)
    images = torch.stack(images)
    labels = torch.tensor([1 if l == task_definitions['B'][1] else 0 for l in labels])
    return images, labels

# Fix: Create support_B before using it
# You need to define task_definitions['B'] and call create_task_datasets for it
# Assuming task_definitions is defined and create_task_datasets is available
# Example of how you would create support_B:
 support_B, query_B = create_task_datasets(train_dataset, task_definitions['B'])

support_loader_B = DataLoader(support_B, batch_size=16, shuffle=True, collate_fn=binary_collate_B)
# Train similarly, evaluate query_B accuracy >95%
# First, train 10-class model on combined train (all tasks A-E)
full_train_idx = []
for task_cls in task_definitions.values():
    cls_indices = [i for i, l in enumerate(train_dataset.targets) if l in task_cls]
    full_train_idx.extend(random.sample(cls_indices, 1000))  # Balanced subsample or full
full_train = Subset(train_dataset, full_train_idx)
full_loader = DataLoader(full_train, batch_size=64, shuffle=True)

model_10c = SimpleCNN().to(device)  # Original 10 outputs
# Train on full_loader with CrossEntropyLoss (10 classes), save 'model_10c.pth'

# Then fine-tune to Binary A
model_10c_binary = BinaryCNN().to(device)
# Copy all except fc2
state_10c = torch.load('model_10c.pth')
model_10c_binary.load_state_dict({k: v for k, v in state_10c.items() if k != 'fc2'}, strict=False)
model_10c_binary.fc2 = nn.Linear(128, 2)  # New random fc2

# Freeze convs if desired, train on support_A as in Phase 2

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 19)

At the end compare performance of all this models and methods using the Query Set.

Also plot the learning curve vs epoch for all the methods

Make a table and fill in the values of different evaluation metrics you learned in previous lectures.

In [ ]:
#Analysis and Plots
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def eval_model(model, loader, device):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            preds = out.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())
    acc = accuracy_score(all_labels, all_preds)
    prec, rec, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average='binary'
    )
    return acc, prec, rec, f1
import matplotlib.pyplot as plt

epochs = range(1, num_epochs+1)
plt.plot(epochs, acc_taskA_large, label='TaskA_large')
plt.plot(epochs, acc_taskA_support, label='TaskA_support_randInit')
plt.plot(epochs, acc_taskB_fromA, label='TaskB_fromA')
plt.plot(epochs, acc_taskA_from10, label='TaskA_from10class')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.title('Learning curves (Accuracy vs Epoch)')
plt.show()


Some Theoritical Questions :

1.   Which strategy in Method 2 works best and why do you feel so ?
2.   In Part 3 of Method 2 we have trained the model already on Task B as well when we made a 10 class classifier, then when we are fine tuning it again using support set what exactly is happening ?
3.   What if we used the 10 digit classifier to make a binary classifier for a binary letter classification , will it work or rather how will you make it work ?
4.   Where exactly have we used Meta Learning, in which approach? Have we even used it ?

---


Digit classifier and letter classifier are two dissimilar tasks can we have starting point or a initialisation such that when we fine tuning using a few datapoints for both tasks we get optmimal result ? This is what we will try to do in MAML ?


---


Think on them sincerely , would love to read your answers!



# ALL THE BEST !